In [20]:
import os
import pandas as pd
import pickle
import numpy as np

map_name = "Town05"
record_duration = 60

# Load the trajectory data
trajectories_dir = f'trajectories/{map_name}_duration_{record_duration}'
trajectory_data = pd.DataFrame(columns=['vehicle_id', 'timestamp', 'x', 'y'])

for filename in os.listdir(trajectories_dir):
    if filename.endswith('.csv'):
        filepath = os.path.join(trajectories_dir, filename)
        df = pd.read_csv(filepath, names=['vehicle_id', 'timestamp', 'x', 'y'])

        trajectory_data = pd.concat([trajectory_data, df], ignore_index=True)

trajectory_data


,vehicle_id,timestamp,x,y
0,1308,<property object at 0x0000013C3B52E4F8>,-123.720955,112.375153
1,1308,<property object at 0x0000013C3B52E4F8>,-123.722900,112.158813
2,1308,<property object at 0x0000013C3B52E4F8>,-123.722900,112.158813
3,1308,<property object at 0x0000013C3B52E4F8>,-123.722900,112.158813
4,1308,<property object at 0x0000013C3B52E4F8>,-123.722900,112.158813
...,...,...,...,...
5180,1392,<property object at 0x0000013C3B52E4F8>,22.524981,194.677338
5181,1392,<property object at 0x0000013C3B52E4F8>,13.676433,194.633545
5182,1392,<property object at 0x0000013C3B52E4F8>,1.120590,194.574799
5183,1392,<property object at 0x0000013C3B52E4F8>,-15.094095,194.567795


In [21]:
# 按时间戳和车辆ID排序
trajectory_data = trajectory_data.sort_values(by=['timestamp', 'vehicle_id'])

grouped = trajectory_data.groupby('timestamp')

filtered_data = []
for timestamp, group in grouped:
    target_vehicle = group.iloc[0]
    x_target, y_target = target_vehicle['x'], target_vehicle['y']

    group = group[group['vehicle_id'] != target_vehicle['vehicle_id']]  # 排除目标车辆
    group = group.assign(distance=np.sqrt((group['x'] - x_target) ** 2 + (group['y'] - y_target) ** 2))
    nearby_vehicles = group[group['distance'] < 20]

    target_info = [int(target_vehicle['vehicle_id']), int(timestamp), float(x_target), float(y_target)]
    filtered_data.append(target_info)

    other_vehicles_info = nearby_vehicles[['vehicle_id', 'timestamp', 'x', 'y']].values.tolist()
    filtered_data.extend(other_vehicles_info)

filtered_df = pd.DataFrame(filtered_data, columns=['vehicle_id', 'timestamp', 'x', 'y'])

filtered_df


ValueError: invalid literal for int() with base 10: '<property object at 0x0000013C3B52E4F8>'